In [49]:
import cv2
import numpy as np

from sklearn.metrics import pairwise


In [50]:
bg = None


ac_weight = 0.60

roi_top = 20
roi_bottom = 350
roi_right = 100
roi_left = 400

In [51]:
def acum_avg(frame, accumumulated_weight):
    global bg 
    if bg is None:
        bg = frame.copy().astype("float")
        return None
    cv2.accumulateWeighted(frame, bg, ac_weight)

In [52]:
def segment(frame, threshold = 40):
    
    dif = cv2.absdiff(bg.astype("uint8"), frame)
    ret, thresholded = cv2.threshold(dif, threshold, 255, cv2.THRESH_BINARY)
    
    contours,hierachy = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) == 0 :
        return None
    else:
        hand_segment = max(contours, key=cv2.contourArea)
        return (thresholded, hand_segment)

In [53]:
def finger_counter(thresholded,  hand_segment):
    
    conv_hull = cv2.convexHull(hand_segment)
    
    #xtream Points
    top    = tuple(conv_hull[conv_hull[:, :, 1].argmin()][0])
    bottom = tuple(conv_hull[conv_hull[:, :, 1].argmax()][0])
    left   = tuple(conv_hull[conv_hull[:, :, 0].argmin()][0])
    right  = tuple(conv_hull[conv_hull[:, :, 0].argmax()][0])
    
    center_x=(left[0] + right[0]) //2
    center_y = (top[1] + bottom[1]) //2
    
    distance = pairwise.euclidean_distances(X=[(center_x, center_y)], Y=[left,right,top,bottom])[0]
    
    max_distance = distance.max()
    
    radius = int(.8* max_distance)
    circumfrence = (2*np.pi * radius)

    circular_roi = np.zeros(thresholded.shape[:2], dtype="uint8")
 
    cv2.circle(circular_roi, (center_x, center_y), radius, 255, 10)
    
   
    circular_roi = cv2.bitwise_and(thresholded, thresholded, mask=circular_roi)

    
    countours,hierarchy = cv2.findContours(circular_roi.copy(),cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    count = 0
    
    for cnt in countours:
        (x,y,w,h) = cv2.boundingRect(cnt)
        
        wrist_position = (center_y+ center_y* .25) > y+h
        limit_points = (circumfrence * 0.25) > cnt.shape[0]
        
        if wrist_position and limit_points :
            
            count+=1
            
            
    return count
        
        

In [54]:
cam = cv2.VideoCapture(0)


num_frames = 0


while True:

    ret, frame = cam.read()

    frame = cv2.flip(frame, 1)

 
    frame_copy = frame.copy()


    roi = frame[roi_top:roi_bottom, roi_right:roi_left]

   
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
  
    gray = cv2.GaussianBlur(gray, (5,5),0)
    
    ###################
    cv2.imshow('lol',gray)

    
    if num_frames < 60:
        acum_avg(gray, ac_weight)
        if num_frames >= 59:
            cv2.putText(frame_copy, " BACKGROUND.", (200, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
         
            
    else:

        hand = segment(gray,25)

    
        if hand is not None:
            
          
            thresholded, hand_segment = hand

          
            cv2.drawContours(frame_copy, [hand_segment + (roi_right, roi_top)], -1, (255, 0, 0),1)

         
            fingers = finger_counter(thresholded, hand_segment)

          
            cv2.putText(frame_copy, str(fingers), (70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

            cv2.imshow("Thesholded_window", thresholded)

    
    cv2.rectangle(frame_copy, (roi_left, roi_top), (roi_right, roi_bottom), (255,255,255), 2)

    
    num_frames += 1

    
    cv2.imshow("Counter window", frame_copy)


   
    k = cv2.waitKey(1) & 0xFF

    if k == 27 :
        break

cam.release()
cv2.destroyAllWindows()